1.0 Reading datasets

Dataset has the below columns:
- amount_tsh - Total static head (amount water available to waterpoint)
- date_recorded - The date the row was entered
- funder - Who funded the well
- gps_height - Altitude of the well
- installer - Organization that installed the well
- longitude - GPS coordinate
- latitude - GPS coordinate
- wpt_name - Name of the waterpoint if there is one
- num_private -
- basin - Geographic water basin
- subvillage - Geographic location
- region - Geographic location
- region_code - Geographic location (coded)
- district_code - Geographic location (coded)
- lga - Geographic location
- ward - Geographic location
- population - Population around the well
- public_meeting - True/False
- recorded_by - Group entering this row of data
- scheme_management - Who operates the waterpoint
- scheme_name - Who operates the waterpoint
- permit - If the waterpoint is permitted
- construction_year - Year the waterpoint was constructed
- extraction_type - The kind of extraction the waterpoint uses
- extraction_type_group - The kind of extraction the waterpoint uses
- extraction_type_class - The kind of extraction the waterpoint uses
- management - How the waterpoint is managed
- management_group - How the waterpoint is managed
- payment - What the water costs
- payment_type - What the water costs
- water_quality - The quality of the water
- quality_group - The quality of the water
- quantity - The quantity of water
- quantity_group - The quantity of water
- source - The source of the water
- source_type - The source of the water
- source_class - The source of the water
- waterpoint_type - The kind of waterpoint
- waterpoint_type_group - The kind of waterpoint


In [19]:
pip install imbalanced-learn scikit-learn matplotlib seaborn


Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install --upgrade imbalanced-learn


Requirement already up-to-date: imbalanced-learn in c:\users\admin\anaconda3\envs\learn-env\lib\site-packages (0.12.4)
Note: you may need to restart the kernel to use updated packages.


In [21]:
# Importing libraries
# Data manipulation and exploration
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Feature selection and engineering
from sklearn.feature_selection import RFE
from imblearn.over_sampling import SMOTE

#Model evaluation
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

## Reading Dataset

1.0 Loading and insecting data

In [22]:

def load_and_inspect_data(file_path):
   
    # Read the dataset
    df = pd.read_csv(file_path)
    
    # Display dataset structure
    print("First 5 rows of the dataset:")
    print(df.head(), "\n")
    
    print("Dataset information:")
    print(df.info(), "\n")
    
    print("Summary statistics for numerical features:")
    print(df.describe(), "\n")
    
    print("Missing values in each column:")
    print(df.isnull().sum(), "\n")
    
    return df

In [23]:
# File paths for our datasets
train_df_file_path = '../data/Train_set_data.csv'
test_df_file_path = '../data/Test_set_data.csv'
train_labels_file_path = '../data/Train_set_labels.csv'

# passing loading and inspecting function to our datasets
train_df = load_and_inspect_data(train_df_file_path)
test_df= load_and_inspect_data(test_df_file_path)
train_labels_df = load_and_inspect_data(train_labels_file_path)

First 5 rows of the dataset:
      id  amount_tsh date_recorded        funder  gps_height     installer  \
0  69572      6000.0    2011-03-14         Roman        1390         Roman   
1   8776         0.0    2013-03-06       Grumeti        1399       GRUMETI   
2  34310        25.0    2013-02-25  Lottery Club         686  World vision   
3  67743         0.0    2013-01-28        Unicef         263        UNICEF   
4  19728         0.0    2011-07-13   Action In A           0       Artisan   

   longitude   latitude              wpt_name  num_private  ... payment_type  \
0  34.938093  -9.856322                  none            0  ...     annually   
1  34.698766  -2.147466              Zahanati            0  ...    never pay   
2  37.460664  -3.821329           Kwa Mahundi            0  ...   per bucket   
3  38.486161 -11.155298  Zahanati Ya Nanyumbu            0  ...    never pay   
4  31.130847  -1.825359               Shuleni            0  ...    never pay   

  water_quality quali

In [24]:
# Merge the train_set and train_set_labels on ID
train_data = pd.merge(train_df,train_labels_df,on='id')
train_data.to_csv('../data/train_data.csv',index=False)
train_data_file_path = '../data/train_data.csv'
train_data = load_and_inspect_data(train_data_file_path)

First 5 rows of the dataset:
      id  amount_tsh date_recorded        funder  gps_height     installer  \
0  69572      6000.0    2011-03-14         Roman        1390         Roman   
1   8776         0.0    2013-03-06       Grumeti        1399       GRUMETI   
2  34310        25.0    2013-02-25  Lottery Club         686  World vision   
3  67743         0.0    2013-01-28        Unicef         263        UNICEF   
4  19728         0.0    2011-07-13   Action In A           0       Artisan   

   longitude   latitude              wpt_name  num_private  ... water_quality  \
0  34.938093  -9.856322                  none            0  ...          soft   
1  34.698766  -2.147466              Zahanati            0  ...          soft   
2  37.460664  -3.821329           Kwa Mahundi            0  ...          soft   
3  38.486161 -11.155298  Zahanati Ya Nanyumbu            0  ...          soft   
4  31.130847  -1.825359               Shuleni            0  ...          soft   

  quality_group

2.0 Handling Missing Values

In [25]:
# Summarize missing values
missing_values_summary = train_data.isnull().sum()
missing_percentage = (missing_values_summary / len(train_data)) * 100
# Create a DataFrame for better visualization
missing_data_summary = pd.DataFrame({
    'Missing Values': missing_values_summary,
    'Percentage': missing_percentage
}).sort_values(by='Missing Values', ascending=False)
# Display columns with missing values
print("Missing Data Summary:")
print(missing_data_summary[missing_data_summary['Missing Values'] > 0])

Missing Data Summary:
                   Missing Values  Percentage
scheme_name                 28166   47.417508
scheme_management            3877    6.526936
installer                    3655    6.153199
funder                       3635    6.119529
public_meeting               3334    5.612795
permit                       3056    5.144781
subvillage                    371    0.624579


In [26]:
# Handle missing data for each column

# scheme_name - Fill with placeholder
train_data['scheme_name'].fillna('Unknown', inplace=True)

# scheme_management - Fill with mode
scheme_mgmt_mode = train_data['scheme_management'].mode()[0]
train_data['scheme_management'].fillna(scheme_mgmt_mode, inplace=True)

# installer and funder - Fill with placeholder
train_data['installer'].fillna('Unknown', inplace=True)
train_data['funder'].fillna('Unknown', inplace=True)

# public_meeting - Fill with mode
public_meeting_mode = train_data['public_meeting'].mode()[0]
train_data['public_meeting'].fillna(public_meeting_mode, inplace=True)

# permit - Fill with mode
permit_mode = train_data['permit'].mode()[0]
train_data['permit'].fillna(permit_mode, inplace=True)

# subvillage - Fill with placeholder
train_data['subvillage'].fillna('Unknown', inplace=True)

# Verifying there are no missing values left
print("Remaining Missing Values:")
print(train_data.isnull().sum().sum())  # Should return 0


Remaining Missing Values:
0


3.0  Columns to drop
- Irrelevant columns 
  - id: Unique identifier, not useful for prediction.
  - recorded_by: Only contains one unique value, so it's not informative
- Redundant columns : other columns contain pretty much similar information
  - region_code: Redundant with region.
  - district_code: Redundant with lga or region.
  - scheme_name: Already handled by scheme_management.
  - extraction_type_group and extraction_type_class: Redundant with extraction_type.
  - quality_group: Redundant with water_quality.
  - quantity_group: Redundant with quantity.
  - source_type and source_class: Redundant with source.
  - waterpoint_type_group: Redundant with waterpoint_type.
- High Cardinality with Low Predictive Power:
  - wpt_name: The name of the waterpoint is unlikely to influence functionality.
  - subvillage: Too granular and high cardinality, unlikely to generalize well.

In [27]:
columns_to_drop = [
    'id', 'recorded_by', 'wpt_name', 'subvillage', 'region_code', 'district_code',
    'scheme_name', 'extraction_type_group', 'extraction_type_class',
    'quality_group', 'quantity_group', 'source_type', 'source_class', 'waterpoint_type_group'
]

train_data.drop(columns=columns_to_drop, inplace=True, errors='ignore')

4.0 Encoding categorical variables

In [28]:
# Identify categorical columns
categorical_columns = train_data.select_dtypes(include=['object', 'category']).columns
print("Categorical Columns:", categorical_columns)

Categorical Columns: Index(['date_recorded', 'funder', 'installer', 'basin', 'region', 'lga',
       'ward', 'scheme_management', 'extraction_type', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quantity', 'source', 'waterpoint_type', 'status_group'],
      dtype='object')


In [29]:
def unique_value_counts(data):
   
    # Select categorical columns
    categorical_columns = data.select_dtypes(include=['object', 'category']).columns

    # Create a summary DataFrame for unique counts
    unique_counts = {
        "Column": categorical_columns,
        "Unique Values": [data[col].nunique() for col in categorical_columns]
    }
    unique_counts_df = pd.DataFrame(unique_counts).sort_values(by="Unique Values", ascending=False)

    print("Unique Value Counts for Categorical Columns:")
    print(unique_counts_df)

    return unique_counts_df

# Call the function on the train_data
categorical_unique_summary = unique_value_counts(train_data)


Unique Value Counts for Categorical Columns:
               Column  Unique Values
2           installer           2145
6                ward           2092
1              funder           1897
0       date_recorded            356
5                 lga            125
4              region             21
8     extraction_type             18
9          management             12
7   scheme_management             12
15             source             10
3               basin              9
13      water_quality              8
11            payment              7
12       payment_type              7
16    waterpoint_type              7
10   management_group              5
14           quantity              5
17       status_group              3


In [30]:
from sklearn.preprocessing import LabelEncoder

def encode_features(data):
   
    # Identify low and high cardinality columns
    low_cardinality_cols = ['payment', 'management', 'water_quality', 
                            'scheme_management','source','basin',
                            'payment_type','waterpoint_type','management_group',
                            'quantity']  
    high_cardinality_cols = ['funder', 'installer','ward','lga','region',
                             'extraction_type','date_recorded']          

    # One-Hot Encoding for low-cardinality columns
    data = pd.get_dummies(data, columns=low_cardinality_cols, drop_first=True)

    # Label Encoding for high-cardinality columns
    label_encoder = LabelEncoder()
    for col in high_cardinality_cols:
        data[col] = label_encoder.fit_transform(data[col])

    return data

# Apply encoding to the train_data DataFrame
train_data = encode_features(train_data)

5.0 Scaling Numerical Features

In [31]:
# Identify numerical columns
numerical_columns = train_data.select_dtypes(include=['int64', 'float64']).columns
print("Numerical Columns:", numerical_columns)

Numerical Columns: Index(['amount_tsh', 'gps_height', 'longitude', 'latitude', 'num_private',
       'population', 'construction_year'],
      dtype='object')


In [32]:
# Initialize the scaler
scaler = StandardScaler()

# Scale the numerical columns
train_data[numerical_columns] = scaler.fit_transform(train_data[numerical_columns])

6.0 Splitting data

In [36]:
# Define features and target
target_column = 'status_group'
X = train_data.drop(columns=[target_column])
y = train_data[target_column]
# splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training Features Shape:", X_train.shape)
print("Testing Features Shape:", X_test.shape)

Training Features Shape: (47520, 88)
Testing Features Shape: (11880, 88)


# Building a dummy model using a dummy classifier

In [41]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize a dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent")  # Predicts the most frequent class
dummy_clf.fit(X_train, y_train)

# Make predictions on the test set
y_dummy_pred = dummy_clf.predict(X_test)

# Evaluate dummy model
print("Dummy Model Accuracy:", accuracy_score(y_test, y_dummy_pred))
print("\nClassification Report for Dummy Model:")
print(classification_report(y_test, y_dummy_pred,zero_division=0))


Dummy Model Accuracy: 0.5435185185185185

Classification Report for Dummy Model:
                         precision    recall  f1-score   support

             functional       0.54      1.00      0.70      6457
functional needs repair       0.00      0.00      0.00       851
         non functional       0.00      0.00      0.00      4572

               accuracy                           0.54     11880
              macro avg       0.18      0.33      0.23     11880
           weighted avg       0.30      0.54      0.38     11880



# Building a Logistic Regression Model

In [46]:
# Initializing logistic regression
from sklearn.metrics import confusion_matrix
# Scale the features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize Logistic Regression model
log_reg = LogisticRegression(max_iter=1000, random_state=42)

# Train the model
log_reg.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = log_reg.predict(X_test_scaled)


Model evaluation

In [47]:
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Accuracy: {accuracy:.2f}")

# Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)


Logistic Regression Accuracy: 0.72

Classification Report:
                         precision    recall  f1-score   support

             functional       0.70      0.89      0.79      6457
functional needs repair       0.46      0.05      0.09       851
         non functional       0.78      0.61      0.68      4572

               accuracy                           0.72     11880
              macro avg       0.65      0.52      0.52     11880
           weighted avg       0.71      0.72      0.70     11880


Confusion Matrix:
[[5779   34  644]
 [ 669   42  140]
 [1782   16 2774]]


In [50]:
# Predict probabilities
y_probs = log_reg.predict_proba(X_test_scaled)

# Calculate ROC-AUC (multi-class, one-vs-rest)
roc_auc = roc_auc_score(y_test, y_probs, multi_class="ovr")
print(f"\nROC-AUC Score: {roc_auc:.2f}")


ROC-AUC Score: 0.81


Results Analysis
1. Accuracy
72% Accuracy: This is an improvement over the dummy model, indicating that Logistic Regression is capturing some patterns in the data.

2. Classification Report
- functional:

Precision: 0.70 → 70% of predicted functional wells are truly functional.
Recall: 0.89 → 89% of actual functional wells are correctly identified.
F1-score: 0.79 → A balance between precision and recall, indicating good performance.

- functional needs repair:

Precision: 0.46 → 46% of predictions for this class are correct.
Recall: 0.05 → Very low recall indicates the model struggles to detect wells needing repair.
F1-score: 0.09 → The model's predictions for this class are quite poor.

- non functional:

Precision: 0.78 → 78% of predicted non-functional wells are truly non-functional.
Recall: 0.61 → The model identifies 61% of actual non-functional wells.
F1-score: 0.68 → Decent performance, though recall could be improved.

3. Macro and Weighted Averages
Macro Avg: Average performance across all classes, unweighted. Lower scores reflect poor performance for the minority class (functional needs repair).
Weighted Avg: Average performance weighted by class support (the number of instances per class). Reflects overall performance better.

4. Confusion Matrix
Diagonal Values (Correct Predictions):
Functional: 5779 out of 6457.
Needs Repair: 42 out of 851.
Non-functional: 2774 out of 4572.

Off-Diagonal Values (Misclassifications):
Many non-functional wells are misclassified as functional (1782 cases).
Very few needs repair wells are identified correctly.

*Key Observations*

Class Imbalance:

The minority class (functional needs repair) is poorly predicted, as Logistic Regression doesn't handle class imbalance well.

Misclassifications:

Significant overlap between functional and non-functional wells, suggesting overlapping feature distributions.